In [58]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [59]:
Sentiments=pd.read_csv("/content/fear_greed_index.csv")
History=pd.read_csv("/content/historical_data.csv")


In [60]:
Sentiments["date"] = pd.to_datetime(Sentiments["date"])
History["Timestamp"] = pd.to_datetime(History["Timestamp"], unit="ms")

In [61]:
History["date"]=History["Timestamp"].dt.date
Sentiments["date"] =Sentiments["date"].dt.date

In [62]:
Sentiments["classification"].unique()


array(['Fear', 'Extreme Fear', 'Neutral', 'Greed', 'Extreme Greed'],
      dtype=object)

In [63]:
mapping = {
    "Extreme Fear": "Fear",
    "Fear": "Fear",
    "Neutral": "Neutral",
    "Greed": "Greed",
    "Extreme Greed": "Greed"
}

Sentiments["sentiment"] = Sentiments["classification"].map(mapping)


In [64]:
Data=History.merge(Sentiments,on="date",how="inner")

**Step 2: Profitability vs Sentiment**

In [65]:
Data.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,...,Order ID,Crossed,Fee,Trade ID,Timestamp,date,timestamp,value,classification,sentiment
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,...,52017706630,True,0.345404,8.950000e+14,2024-10-27 03:33:20,2024-10-27,1730007000,74,Greed,Greed
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,...,52017706630,True,0.005600,4.430000e+14,2024-10-27 03:33:20,2024-10-27,1730007000,74,Greed,Greed
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,...,52017706630,True,0.050431,6.600000e+14,2024-10-27 03:33:20,2024-10-27,1730007000,74,Greed,Greed
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,...,52017706630,True,0.050043,1.080000e+15,2024-10-27 03:33:20,2024-10-27,1730007000,74,Greed,Greed
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,...,52017706630,True,0.003055,1.050000e+15,2024-10-27 03:33:20,2024-10-27,1730007000,74,Greed,Greed


In [66]:
profitability = Data.groupby("sentiment")["Closed PnL"].agg(
    avg_pnl="mean",
    median_pnl="median",
    total_pnl="sum",
    trade_count="count"
).reset_index()

fig = px.bar(
    profitability,
    x="sentiment",
    y=["avg_pnl", "median_pnl"],
    barmode="group",
    title="Average vs Median Closed PnL by Market Sentiment",
    labels={"value": "PnL (USD)", "sentiment": "Market Sentiment"}
)

fig.show()
print(profitability)


  sentiment    avg_pnl  median_pnl     total_pnl  trade_count
0      Fear  50.047622         0.0  6.699925e+06       133871
1     Greed  77.838247         0.0  3.366582e+06        43251
2   Neutral  22.229713         0.0  1.587424e+05         7141


In [67]:
Data["win"] = Data["Closed PnL"] > 0
win_rate = (Data.groupby("sentiment")["win"].mean() * 100).reset_index(name="win_rate")
fig = px.pie(
    win_rate,
    names="sentiment",
    values="win_rate",
    hole=0.5,
    title="Win Rate Share by Market Sentiment"
)
fig.show()
print(win_rate)


  sentiment   win_rate
0      Fear  41.514592
1     Greed  45.349240
2   Neutral  31.718247


**Step 3: Risk Behavior**

In [68]:
avg_size = Data.groupby("sentiment")["Size USD"].mean().reset_index()
fig = px.bar(
    avg_size,
    x="sentiment",
    y="Size USD",
    title="Average Trade Size (USD) by Market Sentiment",
    labels={"Size USD": "Average Trade Size (USD)"}
)
fig.show()

print(avg_size)


  sentiment     Size USD
0      Fear  5259.977837
1     Greed  3581.661513
2   Neutral  3058.848110


In [69]:
size_quantiles = Data.groupby("sentiment")["Size USD"].quantile([0.25, 0.5, 0.75]).reset_index()
fig = px.box(
    size_quantiles,
    x="sentiment",
    y="Size USD",
    title="Trade Size Distribution by Market Sentiment",
    points="outliers"
)

fig.show()

print(size_quantiles)


  sentiment  level_1  Size USD
0      Fear     0.25   202.930
1      Fear     0.50   605.070
2      Fear     0.75  2017.725
3     Greed     0.25   112.630
4     Greed     0.50   545.350
5     Greed     0.75  2222.065
6   Neutral     0.25   179.210
7   Neutral     0.50   554.160
8   Neutral     0.75  1887.850


**Step 4: Trading Volume & Activity**

In [70]:
trade_count = Data.groupby("sentiment").size().reset_index(name="trades")
fig = px.pie(
    trade_count,
    names="sentiment",
    values="trades",
    hole=0.5,
    title="Number of Trades by Market Sentiment"
)

fig.show()
print(trade_count)


  sentiment  trades
0      Fear  133871
1     Greed   43251
2   Neutral    7141


In [71]:
total_volume = Data.groupby("sentiment")["Size USD"].sum().reset_index()
fig = px.bar(
    total_volume,
    x="sentiment",
    y="Size USD",
    color="sentiment",
    title="Total Traded Volume (USD) by Market Sentiment",
    labels={"Size USD": "Total Volume (USD)"}
)

fig.show()
print(total_volume)


  sentiment      Size USD
0      Fear  7.041585e+08
1     Greed  1.549104e+08
2   Neutral  2.184323e+07


In [72]:
active_traders = Data.groupby("sentiment")["Account"].nunique().reset_index(name="Active Traders")
print(active_traders)


  sentiment  Active Traders
0      Fear              32
1     Greed              29
2   Neutral               8


**Step 5: Buy vs Sell Count**

In [73]:
direction_dist = (
    Data.groupby(["sentiment", "Side"])
      .size().reset_index(name="count")
)
fig = px.bar(
    direction_dist,
    y="sentiment",
    x="count",
    color="Side",
    orientation="h",
    title="Directional Bias (Buy vs Sell) by Sentiment"
)
fig.show()

print(direction_dist)


  sentiment  Side  count
0      Fear   BUY  66081
1      Fear  SELL  67790
2     Greed   BUY  18792
3     Greed  SELL  24459
4   Neutral   BUY   3505
5   Neutral  SELL   3636


**Step 6: Risk Per Dollar of Size**

In [74]:
Data["risk_perDollar_size"] = Data["Closed PnL"] / Data["Size USD"]

risk_adjusted = Data.groupby("sentiment")["risk_perDollar_size"].mean().reset_index(name="Size USD")
fig = px.scatter(
    risk_adjusted,
    y="Size USD",
    x="sentiment",
    color="sentiment",
    trendline="ols",
    title="PnL vs Trade Size Colored by Sentiment"
)
fig.update_traces(marker=dict(size=10))
fig.show()

print(risk_adjusted)


  sentiment  Size USD
0      Fear  0.011358
1     Greed  0.054101
2   Neutral  0.015259


**Step 7: Top vs Bottom Trader and its Behavior**

In [75]:
trader_pnl = Data.groupby("Account")["Closed PnL"].sum()

top_traders = trader_pnl[trader_pnl > trader_pnl.quantile(0.75)].index
bottom_traders = trader_pnl[trader_pnl < trader_pnl.quantile(0.25)].index
top_behavior = Data[Data["Account"].isin(top_traders)].groupby("sentiment")["Size USD"].mean()
bottom_behavior = Data[Data["Account"].isin(bottom_traders)].groupby("sentiment")["Size USD"].mean()
top_df = top_behavior.reset_index()
top_df["Trader Group"] = "Top Traders"

bottom_df = bottom_behavior.reset_index()
bottom_df["Trader Group"] = "Bottom Traders"

compare_df = pd.concat([top_df, bottom_df], ignore_index=True)
compare_df
fig = px.line(
    compare_df,
    x="sentiment",
    y="Size USD",
    color="Trader Group",
    markers=True,
    title="Risk-Taking Trend Across Sentiment Regimes"
)

fig.show()

print("Top Traders Avg Size:\n", top_behavior)
print("\nBottom Traders Avg Size:\n", bottom_behavior)



Top Traders Avg Size:
 sentiment
Fear       5517.881231
Greed      4718.220347
Neutral    2728.441554
Name: Size USD, dtype: float64

Bottom Traders Avg Size:
 sentiment
Fear       9956.035311
Greed      3277.497371
Neutral    1079.186118
Name: Size USD, dtype: float64


In [76]:
summary = Data.groupby("sentiment").agg(
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("win", "mean"),
    avg_trade_size=("Size USD", "mean"),
    total_volume=("Size USD", "sum"),
    trades=("Account", "count"),
    traders=("Account", "nunique")
)

summary["win_rate"] *= 100
print(summary)


             avg_pnl   win_rate  avg_trade_size  total_volume  trades  traders
sentiment                                                                     
Fear       50.047622  41.514592     5259.977837  7.041585e+08  133871       32
Greed      77.838247  45.349240     3581.661513  1.549104e+08   43251       29
Neutral    22.229713  31.718247     3058.848110  2.184323e+07    7141        8
